In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os, sys

In [ ]:
labels = ['subway', 'bus', 'car', 'still', 'walking', 'powerChar']

In [ ]:
datapath = '/content/drive/Shareddrives/Project/수집데이터_원본/'
basepath = '/content/drive/Shareddrives/Project/수집데이터_전처리/'

In [ ]:
def make_output_dir(outputpath):
    outputpath_split = outputpath.split(os.path.sep)
    newpath = '/'
    for directory in outputpath_split:
        newpath = os.path.join(newpath, directory)
        if not os.path.exists(newpath):
            os.mkdir(newpath)

In [ ]:
def get_output_filepath(outputpath, position, mode, user):
  filename = f'{position}_{mode}_{user}'

  n = 1
  while n < 1000:
    if n < 10:
      filename_n = filename + f'_00{n}'
    elif n < 100:
      filename_n = filename + f'_0{n}'
    else:
      filename_n = filename + f'_{n}'

    outputfile = os.path.join(outputpath, filename_n)

    if not os.path.exists(outputfile):
      return outputfile
      
    n += 1

In [ ]:
def extract_data(filepath, basepath, user):
  basename = os.path.basename(filepath);

  if 'Sensor' not in basename:
    return

  print(filepath)

  df = pd.read_csv(filepath, encoding='euc-kr')

  mode = df.iloc[0, -2]
  position = df.iloc[0, -1]

  basepath = os.path.join(basepath, position, mode)
  if not os.path.exists(basepath):
    make_output_dir(basepath)

  outputpath = get_output_filepath(basepath, position, mode, user)
  print(outputpath)

  df = df[["GyroX", "GyroY", "GyroZ", "MagX", "MagY", "MagZ", "GraX", "GraY", "GraZ", "LinearAccX", "LinearAccY", "LinearAccZ", "Pressure", "AccX", "AccY", "AccZ"]]

  # pocket, bag의 position인 경우 pocket과 bag에 집어넣는 시간이 존재함
  # 따라서 측정 초반 20초를 날린다
  # 측정 주기가 약 0.017 sec 이므로 0.017 * 1100 하면 20 정도 나옴
  df = df.iloc[1100:]

  df.to_csv(outputpath, sep=' ', header=False, index=False)

# 이미 전처리 한 파일을 제외

In [ ]:
preprocess_log = '/content/drive/Shareddrives/Project/수집데이터_원본/preprocess_log.txt'

In [ ]:
with open(preprocess_log) as f:
  log = f.read()

log = [ file for i, file in enumerate(log.split()) if i % 2 == 0 ]

In [ ]:
print(len(log))

# Confirm이 No인 데이터 있는지 확인

In [ ]:
def check_confirm(filepath):
  basename = os.path.basename(filepath)

  if 'Confirm' not in basename:
    return

  confirm = pd.read_csv(filepath, encoding='euc-kr')

  if confirm.iloc[0, -1] == 'No':
    print(filepath)

In [ ]:
for (path, directory, files) in os.walk(datapath):
  for filename in files:
    check_confirm(os.path.join(path, filename))

# 데이터 전처리 실행

In [ ]:
pass_log = []

In [ ]:
for (path, directory, files) in os.walk(datapath):
  for filename in files:
    file = os.path.join(path, filename)
    if file in log:
      pass_log.append(file)
    else:
      extract_data(file, basepath, path.split(sep='/')[-1])

In [ ]:
print(len(pass_log))